## Testing Dataset

In [1]:
import numpy as np
train = np.loadtxt('ElectricDevices_TRAIN.csv', delimiter=',')
train.shape

(8926, 97)

In [2]:
train_x = train[:, 1:]
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
train_y = train[:, 0] - 1

In [4]:
nclasses = len(np.unique(train_y))
nclasses

7

In [5]:
from keras.utils import np_utils
train_y_c = np_utils.to_categorical(train_y, nclasses)  # convert label to one hot version

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
train_y_c[0]

array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [9]:
test = np.loadtxt('ElectricDevices_TEST.csv', delimiter=',')
test_x = test[:, 1:]
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))
test_y = test[:, 0] - 1
test_y_c = np_utils.to_categorical(test_y, nclasses)

In [10]:
train_x.shape

(8926, 96, 1)

## RNN Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU
from keras.optimizers import RMSprop, SGD
RNN = LSTM  # Use LSTM, you can also use GRU with RNN = GRU
neurons = 64
drop = 0.0
nlayers = 2  # >= 1
impl = 2

model = Sequential()

if nlayers == 1:
    model.add(RNN(neurons, input_shape=(train_x.shape[1], 1), implementation=impl, recurrent_dropout=drop))
else:
    model.add(
        RNN(neurons, input_shape=(train_x.shape[1], 1), implementation=impl, recurrent_dropout=drop, return_sequences=True))
    for i in range(1, nlayers - 1):
        model.add(RNN(neurons, recurrent_dropout=drop, implementation=impl, return_sequences=True))

    model.add(RNN(neurons, recurrent_dropout=drop, implementation=impl))

model.add(Dense(nclasses))
model.add(Activation('softmax'))

############################################
# Training

# optimizer = RMSprop(lr=0.01)
optimizer = SGD(lr=0.01, momentum=0.95)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

epochs = 50
batch_size = 1000
verbose = 1
model.fit(train_x, train_y_c,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_x, test_y_c), verbose=verbose)

Train on 8926 samples, validate on 7711 samples
Epoch 1/50
8926/8926 [==============================] - 34s 4ms/step - loss: 1.9219 - acc: 0.2604 - val_loss: 1.9042 - val_acc: 0.2163
Epoch 2/50
8926/8926 [==============================] - 34s 4ms/step - loss: 1.8498 - acc: 0.2608 - val_loss: 1.8442 - val_acc: 0.2245
Epoch 3/50
8926/8926 [==============================] - 34s 4ms/step - loss: 1.7732 - acc: 0.2601 - val_loss: 1.8123 - val_acc: 0.2219
Epoch 4/50
8926/8926 [==============================] - 33s 4ms/step - loss: 1.7280 - acc: 0.2579 - val_loss: 1.8026 - val_acc: 0.2238
Epoch 5/50
8926/8926 [==============================] - 33s 4ms/step - loss: 1.6943 - acc: 0.3638 - val_loss: 1.7942 - val_acc: 0.3500
Epoch 6/50
8926/8926 [==============================] - 34s 4ms/step - loss: 1.6641 - acc: 0.4022 - val_loss: 1.7909 - val_acc: 0.3514
Epoch 7/50
8926/8926 [==============================] - 33s 4ms/step - loss: 1.6330 - acc: 0.4071 - val_loss: 1.7782 - val_acc: 0.3544
Epoch 8

## Evaluation

In [ ]:
score, acc = model.evaluate(test_x, test_y_c, batch_size=batch_size, verbose=0)

print()
print('ACC= ', acc)
print()
test_pred = model.predict_classes(test_x, verbose=0)
print('Confusion Matrix')
print('-'*20)
print(confusion_matrix(test_y, test_pred))
print()
print('Classification Report')
print('-'*40)
print(classification_report(test_y, test_pred))
print()
print("Ending:")